In [ ]:
from visualize_data import json_to_dataframe
import re
import pandas as pd
import numpy as np

In [ ]:
df = json_to_dataframe("js")

In [ ]:
# 레벨별

def multi_index_to_dict(data):
    temp_dict = dict()
    for (index_1,index_2), value in data.items():
        if index_1 not in temp_dict:
            temp_dict[index_1] = {}
        temp_dict[index_1][index_2]  = value
    return temp_dict


def get_level_dict(dataframe):
    
    # 레벨별 문제 수
    temp_df = df.copy()
    temp_df["problem_name"] = temp_df["problem_name"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby("level")["problem_name"].nunique()
    level_pname = temp_df.to_dict()

    # 레벨별 고유한 문제 유형 개수
    temp_df = df.copy()
    temp_df["problem_type"] = temp_df["problem_type"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby("level")["problem_type"].nunique()
    level_ptype = temp_df.to_dict()
    
    #레벨멸 메소드 이용 수, distinct value
    temp_df = df.copy()
    temp_df["file_name"] = temp_df["file_name"].apply(lambda x: re.sub("\s", "", x))
    temp_df["problem_type"] = temp_df["problem_type"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby(["level","problem_type"])["file_name"].nunique()
    level_dcnt_method = multi_index_to_dict(temp_df)
    
    # 레벨별 메소드 이용수, 누적
    temp_df = df.groupby(['level','function_method'])['countmethod'].sum()
    level_cnt_method = multi_index_to_dict(temp_df)

    # 메소드 이용 현황, 누적 값
    method_cnt = df.groupby("function_method")["countmethod"].sum().sort_values(ascending = False).to_dict()
    
    return level_pname, level_ptype, level_dcnt_method, level_cnt_method, method_cnt



In [ ]:
# 연도별



In [ ]:
# 출제업체별



In [ ]:
# 문제유형별
def get_ptype_dict(df):
    '''
    JavaScript 문제의 문제유형별 문제 수를 카운트하는 함수입니다.
    JavaScript 문제의 문제유형마다 사용된 메소드별 횟수를 카운트하는 함수입니다.
    '''

    # 문제의 문제유형별 문제 수

    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates(['file_name'], keep='first')
    temp_df = temp_df['problem_type'].value_counts().to_frame()

    ptype_name = temp_df.to_dict()["problem_type"]


    # 문제유형별 사용된 메소드별 횟수
    temp_df = df.copy()
    temp_df = temp_df[temp_df["function_method"] != 0]
    temp_df = temp_df.groupby(['problem_type', 'function_method'], as_index=False)['countmethod'].sum()
    temp_df = temp_df.pivot(index='problem_type', columns='function_method', values='countmethod')

    ptype_method = temp_df.to_dict()
    
    return ptype_name, ptype_method

In [ ]:
# 메소드_모듈별



In [ ]:
# 전체 데이터 저장하는 모듈

def get_dict(df):
    '''
    JavaScript 문제의 전체 데이터를 dictionary 형태로 변환하여 반환하는 함수입니다.
    '''    

    # 전체 데이터 저장할 딕셔너리
    temp_dict = {}
    
    # 딕셔너리에 파트별 데이터 삽입
    
    level_pname, level_ptype, level_dcnt_method, level_cnt_method, method_cnt = get_level_dict(df)  # 레벨별 & 전체적인 메서드 사용 현황
    ptype_name, ptype_method = get_ptype_dict(df)  # 문제유형별

    # level 별
    temp_dict['level_pname'] = level_pname
    temp_dict['level_ptype'] = level_ptype
    temp_dict['level_dcnt_method'] = level_dcnt_method
    temp_dict['level_cnt_method'] = level_cnt_method
    temp_dict['method_cnt'] = method_cnt

    # 문제유형별
    temp_dict['ptype_name'] = ptype_name
    temp_dict['ptype_method'] = ptype_method
    
    return temp_dict

In [ ]:
# main.py 에서 실행될 코드

# Standard library import
import json

# Local application library
from visualize_data import json_to_dataframe
from visualize_data import py
from visualize_data import js

py_df = json_to_dataframe('py')
js_df = json_to_dataframe('js')

temp_dict = {}
temp_dict['py'] = py.get_dict(py_df) # py.py 모듈의 get_dict
temp_dict['js'] = js.get_dict(js_df) # js.py 모듈의 get_dict

with open("chart_data.json", "w", encoding="utf-8") as f:
    json.dump(temp_dict, f, ensure_ascii=False, indent=4)